In [1]:
# kaggle titanic
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# dataset
DATA_DIR = './data/'
TRAIN_DATA = 'train.csv'
TEST_DATA = 'test.csv'
SUBMIT_DATA = 'gender_submission.csv'

In [3]:
# load train data
RAW_DATA = pd.read_csv(os.path.join(DATA_DIR, TRAIN_DATA))

In [4]:
data_test = pd.read_csv(os.path.join(DATA_DIR, TEST_DATA)) 

In [5]:
data = RAW_DATA

In [6]:
# fillna
data['Fare'] = data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

data.isnull().sum()

# Objective variable and Explanatory variable
y_1 = data['Survived']
X_1 = data.drop(columns={'Survived', 'Name', 'Ticket', 'Embarked', 'Cabin'})
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_1_le = X_1
Sex_le = le.fit_transform(X_1_le['Sex'])
X_1['Sex'] = Sex_le
X_1 = X_1.fillna(0)

In [7]:
X_train1, X_test1, y_train, y_test = train_test_split(X_1, y_1, random_state=0, test_size=0.3)

In [8]:
# fillna
data_test['Fare'] = data_test['Fare'].fillna(np.mean(data_test['Fare']), inplace=True)

# Objective variable and Explanatory variable
# y_1 = data['Survived']
data_test = data_test.drop(columns={'Name', 'Ticket', 'Embarked', 'Cabin'})
data_test_le = data_test
data_test_sex_le = le.fit_transform(data_test_le['Sex'])
data_test_le['Sex'] = data_test_sex_le
data_test_le = data_test_le.fillna(0)
data_predict = data_test_le

In [10]:
# グリッドサーチ
# param_grid = {'C':[0.1,1,10,100,1000], 'gamma': [1,0.1,0.001,0.0001,0.5]}
param_grid = {'C': [0.01, 0.5, 1, 100, 500], 'max_iter': [300, 500, 1000]}
grid = GridSearchCV(LogisticRegression(),param_grid, verbose=1)

grid.fit(X_train1, y_train)

grid.best_params_

Fitting 5 folds for each of 15 candidates, totalling 75 fits


{'C': 0.5, 'max_iter': 300}

In [11]:
# modeling
# Logistic Regression2
# lr1 = LogisticRegression(penalty='l2', solver='sag', random_state=0, max_iter=3000)
lr3 = LogisticRegression(random_state=0, max_iter=300, C=0.5)

In [13]:
# 学習
lr3.fit(X_train1, y_train)

# 予測
lr_pred03 = lr3.predict(data_predict)

lr3.score(X_train1, y_train), lr3.score(X_train1, y_train)

(0.7929373996789727, 0.7929373996789727)

In [35]:
# GridsearchCV
# パラメータを dict 型で指定
param_grid = {'max_iter' : [100, 300], 'C': [0.5, 0.6]}

# validation set は GridSearchCV が自動で作成してくれるため，
# training set と test set の分割のみを実行すればよい
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train1, y_train, random_state=0)

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)

# fit 関数を呼ぶことで交差検証とグリッドサーチがどちらも実行される
grid_search.fit(X_train2, y_train2)

print('Test set score: {}'.format(grid_search.score(X_test2, y_test2)))
print('Best parameters: {}'.format(grid_search.best_params_))
print('Best cross-validation: {}'.format(grid_search.best_score_))

/data_01/script/script_python/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/data_01/script/script_python/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

Test set score: 0.8012820512820513
Best parameters: {'C': 0.5, 'max_iter': 300}
Best cross-validation: 0.7817890642873484


In [37]:
lr4 = LogisticRegression(random_state=0, C=0.5, max_iter=300)

# 学習
lr4.fit(X_train2, y_train2)

# 予測
lr_pred04 = lr4.predict(data_predict)

print(lr4.score(X_train2, y_train2))

In [43]:
lr4.score(X_train2, y_train2)

0.8008565310492506

In [39]:
data_submit = pd.read_csv(os.path.join(DATA_DIR, SUBMIT_DATA))

In [40]:
# 提出用ファイルに予測結果を連結
data_submit['Survived'] = lr_pred04

In [22]:
# data_submit.shape

(418, 2)

In [41]:
data_submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [42]:
# write csv
data_submit.to_csv(os.path.join(DATA_DIR, 'submission_lr04.csv'), index=False)